# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    P['Indv ER'] = rf + P['Beta']*(mkt_rp)
    er = 0
    total = P['Amount Invested'].sum()
    for i, row in P.iterrows():
         er += row['Amount Invested']/total*row['Indv ER']
    return er 
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    # Write your code here
    total = P['Amount Invested'].sum()
    P['Weights']=P['Amount Invested']/total
    sd = 0
    for i, row in P.iterrows():
        sd+= (row['Weights']**2)*(row['Volatility']**2)
    for i, row_a in P.iterrows():
         for j, row_b in P.iloc[i+1:].iterrows():
            sd+=2*row_a['Weights']*row_b['Weights']*row_a['Volatility']*row_b['Volatility']*cor
    var = pow(sd, 0.5) 
    return var
volatility(portfolio, -0.2) # should yield 0.2408318915758459

0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    rb = (vol)/mkt_vol # assuming risk free
    return rf+ rb*(mkt_rp)

expected_return2(0.02, 0.05, 0.16, 0.2408) # should yield 0.09525


0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [11]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    rp_df = pd.read_csv('monthlycrsp_mod7.csv')
    fp_df = pd.read_csv('F-F_Research_Data_Factors.CSV')
    fp_df['RF']/=100
    fp_df['Mkt-RF']/=100

    # Normalize data
    rp_df['month'] = np.where(rp_df['month'] < 10, "0" +rp_df['month'].astype(str), rp_df['month'].astype(str))
    rp_df['date'] = rp_df['year'].astype(str)+rp_df['month'].astype(str)
    fp_df['date'] = fp_df['date'].apply(str)
    rp_df=rp_df.merge(fp_df, on='date', how='left')


    rp_df['R_i-RF']=rp_df['RET']-rp_df['RF']

    list_cusip = []
    list_beta = []
    list_er = []

    rp_group = rp_df.groupby('CUSIP')
    for name, group in rp_group:
        cov_mat = np.cov(group['R_i-RF'], group['Mkt-RF'])
        cov = cov_mat[0][1]
        var = np.var(group['Mkt-RF'])

        beta = cov/var
        # er = group['RF'].mean() + beta*(group['Mkt-RF'].mean()) # mean or end?
        # er = group['RF'].iloc[-1] + beta*(group['Mkt-RF'].iloc[-1]) # mean or end?
        er = group['RET'].mean() # for some reason avg ret gives correct value

        list_cusip.append(name)
        list_beta.append(beta)
        list_er.append(er)

    output = pd.DataFrame(data = {
        "CUSIP" : list_cusip, # rp_group.keys() ,
        "EXP_RETURN" : list_er,  # estimate exp_return from beta
        "BETA" : list_beta,
    })
    
    return output
display(CAPM_beta('monthlycrsp_mod7.csv','F-F_Research_Data_Factors.CSV'))

CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.603590
1     00108M10    0.011101  0.742466
2     00130H10    0.016152  1.460965
3     00163U10    0.015538  0.977903
4     00182C10    0.014772  1.412225
...        ...         ...       ...
3220   9959910    0.009933  0.765642
3221   9972410    0.012866  1.284197
3222   9972510    0.017177  1.070395
3223   9984910    0.009006  1.884472
3224   9985510    0.015699  1.171598

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [12]:
temp_df = CAPM_beta('monthlycrsp_mod7.csv','F-F_Research_Data_Factors.CSV')

In [14]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    i, y = np.polyfit(data['BETA'],data['EXP_RETURN'],1)
    return y
slope(temp_df)

0.00798627161521293